Clean Landsat composites created with the file "download_landsat_composite.ipynb"

1. remove high values (1.000000e+20) and replace with NAN
2. add spectral indices

In [1]:
# Import general libraries
import rasterio
import numpy as np
import rasterio

from pathlib import Path

# Import EOdal libraries
from eodal.core.raster import RasterCollection

# define the year
year = 2021 


# Path to the raw input TIFF file
input_tif_path = f'S:\MSc_23_TimckeFinn\data\EOdal\landsat_median_composite_{year}_smallAOI.tif'

# Path to the clean output TIFF file
output_tif_path = f'S:\MSc_23_TimckeFinn\data\EOdal\landsat_median_composite_{year}_smallAOI_clean.tif'

# Open the TIFF file using rasterio
with rasterio.open(input_tif_path) as src:

    # Read all bands of the raster data
    raster_data = src.read()

    # Replace specific values (e.g., 1e+20) with NaN for each band
    replaced_data = np.where(raster_data == 1e+20, np.nan, raster_data)

    # Create a new raster file with NaN values
    with rasterio.open(
        output_tif_path,
        'w',
        driver='GTiff',
        height=src.height,
        width=src.width,
        count=src.count,  # Number of bands
        dtype=replaced_data.dtype,
        crs=src.crs,
        transform=src.transform,
    ) as dst:
        dst.write(replaced_data)

# read in clean tif again and add some SI

#read tif from disk:
fpath_raster = Path(f'S:\MSc_23_TimckeFinn\data\EOdal\landsat_median_composite_{year}_smallAOI_clean.tif')

# initialize a RasterCollection from the .tif file
composite = RasterCollection.from_multi_band_raster(fpath_raster=fpath_raster,
                                                    band_names_dst= ["blue_median", "green_median", "red_median", "nir08_median", "swir16_median","swir22_median"],
                                                    band_aliases = ["blue", "green", "red", "nir_1", "swir_1","swir_2"])

# calculate some SI
composite.calc_si('NDVI', inplace = True)
composite.calc_si('NDWI', inplace = True)
composite.calc_si('EVI', inplace = True)
composite.calc_si('CI_GREEN', inplace = True)
composite.calc_si('MSAVI', inplace = True)
composite.calc_si('BSI', inplace = True)

# save as GeoTiff
composite.to_rasterio(f'S:\MSc_23_TimckeFinn\data\EOdal\landsat_median_composite_{year}_smallAOI_clean_si.tif')


In [2]:
composite.band_summaries()

,min,mean,std,max,count,geometry,band_name
0,0.077440,0.080810,0.001985,0.095985,26928.0,"POLYGON ((874635.000 54285.000, 874635.000 582...",blue_median
1,0.082880,0.089742,0.004197,0.107940,26928.0,"POLYGON ((874635.000 54285.000, 874635.000 582...",green_median
2,0.078880,0.084373,0.003764,0.114200,26928.0,"POLYGON ((874635.000 54285.000, 874635.000 582...",red_median
3,0.087570,0.180042,0.024032,0.248670,26928.0,"POLYGON ((874635.000 54285.000, 874635.000 582...",nir08_median
4,0.078815,0.123620,0.014078,0.183300,26928.0,"POLYGON ((874635.000 54285.000, 874635.000 582...",swir16_median
5,0.076170,0.095905,0.007857,0.151620,26928.0,"POLYGON ((874635.000 54285.000, 874635.000 582...",swir22_median
6,0.018176,0.355873,0.069525,0.494187,26928.0,"POLYGON ((874635.000 54285.000, 874635.000 582...",NDVI
7,-0.455743,-0.328980,0.067513,0.002109,26928.0,"POLYGON ((874635.000 54285.000, 874635.000 582...",NDWI
8,0.008038,0.220245,0.050080,0.359439,26928.0,"POLYGON ((874635.000 54285.000, 874635.000 582...",EVI
9,-0.004209,1.003759,0.234130,1.674734,26928.0,"POLYGON ((874635.000 54285.000, 874635.000 582...",CI_GREEN
